In [1]:
from fastplotlib.widgets import ImageWidget
import numpy as np
import tifffile

# Single image sequence

In [2]:
# using Sue demo movie
path = "/home/kushal/caiman_data/example_movies/Sue_2x_3000_40_-46.tif"
a = tifffile.imread(path)

In [3]:
iw = ImageWidget(
    data=a, 
    vmin_vmax_sliders=True,
    cmap="gnuplot2"
)

RFBOutputContext()

In [4]:
iw.show()

# Try scrolling along the "t" - time dimension and changing the min-max!

### Play with setting different window functions

These can also be given as kwargs to `ImageWidget` during instantiation

In [5]:
# must be in the form of {dim: (func, window_size)}
iw.window_funcs = {"t": (np.mean, 5)}

In [6]:
# change the winow size
iw.window_funcs["t"].window_size = 23

In [7]:
# change the function
iw.window_funcs["t"].func = np.max

In [8]:
# or set it again
iw.window_funcs = {"t": (np.min, 11)}

### `ImageWidget` embeds a `Plot` or `GridPlot` that is accessible using `ImageWidget.plot`

In [9]:
iw.plot

unnamed: Plot @ 0x7f4e03f26290
  parent: None
  Graphics:
	'image': ImageGraphic @ 0x7f4e03f73010

In [10]:
# graphics etc. just like before

In [11]:
iw.plot.graphics

('image': ImageGraphic @ 0x7f4e03f73010,)

In [12]:
iw.plot.graphics[0].cmap="gray"

# Close the canvas

In [13]:
iw.plot.canvas.close()

# Gridplot of txy data

In [14]:
dims = (100, 512, 512)
data = [np.random.rand(*dims) for i in range(4)]

In [15]:
iw = ImageWidget(
    data=data, 
    vmin_vmax_sliders=True,
    names=["zero", "one", "two", "three"],
    window_funcs={"t": (np.mean, 5)},  # you can pass window_funcs as a kwarg
    cmap="gnuplot2", 
)

RFBOutputContext()

### pan-zoom controllers are all synced in a `ImageWidget`

In [16]:
iw.show()

### This `ImageWidget` contains a `GridPlot`, so you can index the subplots using the names given to `ImageWidget`

In [17]:
iw.plot["two"]

two: Subplot @ 0x7f4e03ffb9d0
  parent: None
  Graphics:
	'image': ImageGraphic @ 0x7f4de4336350

### change window functions just like before

In [18]:
iw.window_funcs["t"].func = np.max

In [19]:
# make sure to close plot before moving to the next one
iw.plot.canvas.close()

## Gridplot of volumetric data

In [20]:
dims = (256, 256, 5, 100)
data = [np.random.rand(*dims) for i in range(4)]

iw = ImageWidget(
    data=data, 
    slider_dims=["t", "z"], 
    dims_order="xyzt", # example of how you can set this for non-standard orders
    vmin_vmax_sliders=True,
    names=["zero", "one", "two", "three"],
    # window_funcs={"t": (np.mean, 5)}, # window functions can be slow when indexing multiple dims
    cmap="gnuplot2", 
)

RFBOutputContext()

In [21]:
iw.show()

### window functions, can be slow when you have "t" and "z"

In [22]:
iw.window_funcs = {"t": (np.mean, 11)}

In [23]:
# make sure to close plot and kill kernel
iw.plot.canvas.close()